Review the data and the schema

Create the tables

In [5]:
SET NOCOUNT ON;

USE tempdb;
-- Create a new table called '[ParentOf]' in schema '[dbo]'
-- Drop the table if it already exists
IF OBJECT_ID('[dbo].[ParentOf]', 'U') IS NOT NULL
DROP TABLE [dbo].[ParentOf]
GO

-- Create the table in the specified schema
CREATE TABLE [dbo].[ParentOf]
(
    Parent VARCHAR(255) NOT NULL,
    Child VARCHAR(255) NOT NULL,
    BirthYear INT,
    CHECK (Parent <> Child)
);
GO
 
CREATE UNIQUE INDEX idx_unc_parent_child
  ON dbo.ParentOf(Parent, Child)
  INCLUDE(BirthYear);
GO

INSERT INTO [dbo].[ParentOf] VALUES ('Alice', 'Carol', 1945);
INSERT INTO [dbo].[ParentOf] VALUES ('Bob', 'Carol', 1945);
INSERT INTO [dbo].[ParentOf] VALUES ('Carol', 'Dave', 1970);
INSERT INTO [dbo].[ParentOf] VALUES ('Carol', 'George', 1972);
INSERT INTO [dbo].[ParentOf] VALUES ('Dave', 'Mary', 2000);
INSERT INTO [dbo].[ParentOf] VALUES ('Eve', 'Mary', 2000);
INSERT INTO [dbo].[ParentOf] VALUES ('Mary', 'Frank', 2020);

: Msg 40508, Level 16, State 1, Line 3
USE statement is not supported to switch between databases. Use a new connection to connect to a different database.

: Msg 229, Level 14, State 5, Procedure DDL_Trigger_Azure_sample, Line 13
The INSERT permission was denied on the object 'DDLEvemnts', database 'master', schema 'dbo'.

: Msg 1088, Level 16, State 12, Line 20
Cannot find the object "dbo.ParentOf" because it does not exist or you do not have permissions.

: Msg 208, Level 16, State 1, Line 25
Invalid object name 'dbo.ParentOf'.

Total execution time: 00:00:00.505

In [6]:
USE tempdb;
SELECT TOP (1000) [Parent]
      ,[Child]
      ,[BirthYear]
  FROM [dbo].[ParentOf]

: Msg 40508, Level 16, State 1, Line 1
USE statement is not supported to switch between databases. Use a new connection to connect to a different database.

Total execution time: 00:00:00.111

Find the parents of Frank, then the parents of the parents until there are no more

In [7]:
USE tempdb;
go

with Ancesstor as
    (
        Select Parent as p
        from [dbo].[ParentOf]
        where Child='Frank'

        UNION ALL

            SELECT Parent
            from Ancesstor, [dbo].[ParentOf]
            WHERE Ancesstor.p = [dbo].[ParentOf].[Child]
    )
SELECT *
FROM Ancesstor;

: Msg 40508, Level 16, State 1, Line 1
USE statement is not supported to switch between databases. Use a new connection to connect to a different database.

: Msg 208, Level 16, State 1, Line 4
Invalid object name 'dbo.ParentOf'.

Total execution time: 00:00:00.224

In [8]:
USE tempdb;
go

WITH Descendant
AS (
    SELECT parent + ' -> ' + child as lineage, child as c, birthyear, 0 as parentAge
    FROM [dbo].[ParentOf] WHERE parent = 'Alice'

    UNION ALL

    SELECT parent + ' -> ' + child as lineage, child as c, [dbo].[ParentOf].birthyear, [dbo].[ParentOf].birthyear - Descendant.birthyear
    FROM Descendant, [dbo].[ParentOf]
    WHERE Descendant.c = [dbo].[ParentOf].parent
    )

SELECT lineage, birthYear, parentAge from Descendant

: Msg 40508, Level 16, State 1, Line 1
USE statement is not supported to switch between databases. Use a new connection to connect to a different database.

: Msg 208, Level 16, State 1, Line 4
Invalid object name 'dbo.ParentOf'.

Total execution time: 00:00:00.222

Find the number of generation below a parent

In [24]:
USE tempdb;
go

DECLARE @root AS INT = 1945;
DECLARE @maxlevel AS INT = 4;
 
WITH C AS
(
  SELECT [Parent]
        ,[Child]
        ,[BirthYear]
        ,0 as generations_below
  FROM [dbo].[ParentOf]
  WHERE BirthYear = (SELECT MAX(BirthYear) FROM [dbo].[ParentOf]) 

  UNION ALL

  SELECT S.[Parent]
        ,S.[Child]
        ,S.[BirthYear]
        ,M.generations_below + 1
  FROM C as M
  INNER JOIN [dbo].[ParentOf] AS S
  on S.Child = M.Parent
  AND M.generations_below < @maxlevel
)
SELECT Parent, Child, BirthYear, generations_below
FROM C
ORDER by BirthYear, generations_below
OPTION (MAXRECURSION 15);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.143

Parent,Child,BirthYear,generations_below
Alice,Carol,1945,3
Bob,Carol,1945,3
Carol,Dave,1970,2
Dave,Mary,2000,1
Eve,Mary,2000,1
Mary,Frank,2020,0


Cleanup

In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[ParentOf]') AND type in (N'U'))
DROP TABLE [dbo].[ParentOf]
GO